In [23]:
from IPython.display import Markdown as md


In [24]:
try:
    run_id = snakemake.wildcards.run_id
except:
    run_id = snakemake.wildcards.dataset_id
md(f'# Report {run_id} {snakemake.wildcards.dataset_id} aligned to {snakemake.wildcards.refgenome_id}')

In [3]:
import pandas as pd
from pathlib import Path 
import re

from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np


In [4]:
##plot settings
sns.set(font_scale=1.4)
sns.set_style("whitegrid")

In [5]:
## -- Functions: -- ##

## RGB
ont_colors_rgb = [
    (0,132,169),
    (144,198,232),
    (69,85,96),
    (0,98,113),
    (217,217,214),
    (191,184,175)
]

def rgb2hex(color):
    return "#{0:02x}{1:02x}{2:02x}".format(*(color))

def ax_settings(ax, dejoined=False) -> None:
    
    def _settings(ax):
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_edgecolor('#444444')
        ax.spines['bottom'].set_linewidth(2)
        ax.spines['left'].set_edgecolor('#444444')
        ax.spines['left'].set_linewidth(2)
    
    if dejoined:
        ax.spines['left'].set_position(('outward', 10))
        ax.spines['bottom'].set_position(('outward', 10))
    
    if isinstance(ax, sns.FacetGrid):
        for a in ax.axes.flat:
            _settings(a)
    else:
        _settings(ax)

def N50(a, perc=0.5):
    a = np.array(a)
    a[::-1].sort() ## sort in deascending order
    csum = a.cumsum()
    total = csum.max() 
    nx_idx = np.where(csum == csum[csum>=(total*perc)].min())
    return a[nx_idx][0]

def ncrf_agg_stats(x):
    d = {
        'total_reads': len(x),
        'telo_len_min': x['telomere_len'].min(),
        'avg_telo_len': x['telomere_len'].mean(),
        'median_telo_len': x['telomere_len'].median(),
        'telo_N50': N50(x['telomere_len']),
        'telo_len_max': x['telomere_len'].max(),

        'avg_subtelomere_len': x['subtelomere_len'].mean(),
        'read_len_min': x['read_len'].min(),
        'avg_read_len': x['read_len'].mean(),
        'read_N50': N50(x['read_len']),
        'read_len_max': x['read_len'].max(),
        
        'num_read_gt15k': len(x[x['read_len']>15_000]),
        'avg_motif_indentify': x['ident'].mean(),
        'num_pass_filter': len(x[x['pass_filter']])

    }
    sf = pd.Series(d)
#     sf['num_telomeres_runs'] = sf.total_reads / sf.num_runs
    return sf.round(2)

def basecall_stats(x):
    d = {
        'total_reads': len(x),
        'total_bases_mb': x.sum()/1e6,
        'min_read_len': x.min(),
        'mean_read_len': x.mean(),
        'median_read_len': x.median(),
        'N50_read_len': N50(x),
        'max_read_len': x.max(),
        'num_gt_15k': len(x[x>15_000])
        
    }
    return pd.Series(d).round(3)


## Set colors
RED = "#b53535"
ONT_COLOR = list(map(rgb2hex, ont_colors_rgb)) + [RED]
sns.set_palette(ONT_COLOR)

In [22]:
## Directories 
NCRF_F = snakemake.input.ncrf
READ_STATS_F = snakemake.input.bcalls
BAM_F = snakemake.input.bam

# FLOWCELL = snakemake.wildcards.run_id 
FLOWCELL =  run_id

OUTPUT_F = snakemake.output[0].replace('.basecall_stats.csv', '')

In [10]:

## import ncrf data
ncrf = pd.read_csv(NCRF_F)#.drop(['#line', 'motif'], axis=1)
ncrf['read_id2']  = ncrf.read_id
ncrf['flowcell'] = FLOWCELL
ncrf['frag_id'] = ncrf.read_id2.apply(lambda x: x.split('/')[-1])
ncrf['read_id'] = ncrf.read_id2.apply(lambda x: x.split('/')[0])
ncrf['pass_filter'] = ncrf[['has_subtelomere','started_correctly','is_terminal']].apply(lambda x: all(x), axis=1)

## Check for Split reads 
mulitsegment_reads = ncrf.read_id.value_counts().sort_values(ascending=False)
total_telomeric_reads = len(mulitsegment_reads)
mulitsegment_reads = mulitsegment_reads.loc[mulitsegment_reads>1]
ncrf = ncrf.query('pass_filter == True').copy() ## filter out reads that do not pass filter


In [11]:
## import Read stats for all reads:
read_stats = (
    pd
    .read_csv(READ_STATS_F)
    .rename(
        columns={'#id':"read_id", 'length':'read_len'}
    )
)
read_stats.loc[:,'is_telomeric'] = 'not_telomeric'
read_stats.loc[read_stats.read_id.isin(ncrf.read_id),'is_telomeric'] = 'telomeric'
read_stats.to_csv(OUTPUT_F+'.read_stats.csv', index=False)

## Basecall Stats
bs_stats = read_stats.groupby('is_telomeric')['read_len'].apply(basecall_stats).unstack(0)
bs_stats['total'] = bs_stats.sum(1).round(3)
bs_stats.loc["min_read_len":"max_read_len", 'total'] = pd.NA
bs_stats['percentage'] = (bs_stats.telomeric / bs_stats.total).mul(100)
bs_stats.to_csv(snakemake.output[0], index=True)
md(f"## Basecall stats for all data (No filter):\n{bs_stats.to_markdown()}")


In [12]:
# TODO: # fix telomere stats
ncrf_agg  = ncrf.groupby('flowcell').apply(ncrf_agg_stats).T
ncrf_agg.to_csv(OUTPUT_F+'.ncrf.agg.csv',index=True)

# md(f"### Number of reads with Chimeric reads {100 *len(mulitsegment_reads)/total_telomeric_reads:.3f} % ({len(mulitsegment_reads)}/{total_telomeric_reads})")
md(data=f"""
# Telomoere stats (filtered, informatically digested)\n{ncrf_agg.to_markdown()}

A some portion of reads could ligate to each other, denote here a "Split" (Chimeric) reads
* Number of Split reads **{100 *len(mulitsegment_reads)/total_telomeric_reads:.3f}% ({len(mulitsegment_reads)}/{total_telomeric_reads})**


""")

In [26]:
bam_df = pd.read_csv(BAM_F)
chromosome_order = [f'chr{x}' for x in range(1,23)] + ['chrX', 'chrY']
bam_df['chrom_name'] = bam_df.rname.apply(lambda x: x.split('/')[0].split('.')[0])
bam_df['arm'] = bam_df.rname.apply(lambda x: x.split('/')[0].split('.')[1])


# bam_df['chrom_name'] = bam_df.arm.apply(lambda x: x.split('.')[0])
# bam_df['arm'] = bam_df.arm.apply(lambda x: x.split('.')[1])

bam_df['chromosomes'] = pd.Categorical(
    bam_df.chrom_name, categories=chromosome_order, ordered=True)
length_before = len(bam_df)
bam_df = bam_df.merge(ncrf[['read_id2', 'telomere_len']], left_on='read_id', right_on='read_id2')

assert len(bam_df) == length_before
# bam_df
bam_df.to_csv(OUTPUT_F+'.bam_stats.csv',index=False)


In [27]:
ncrf.loc[:, 'is_mapped'] ='unmapped'
ncrf.loc[ncrf.read_id2.isin(bam_df.read_id),'is_mapped'] = 'mapped'
ncrf.to_csv(OUTPUT_F+'.ncrf.csv', index=False)


In [28]:

fig, ax = plt.subplots(figsize=(8,5))

ax_settings(
    ncrf.loc[ncrf.pass_filter] ## pass filter reads
    .is_mapped
    .value_counts(normalize=True)
    .mul(100)
    .plot(
        kind='barh', 
        stacked=True, 
        title='Percentage telomeres mapped', 
        xlim=(0,100),
        color=ONT_COLOR[0],
        ax=ax
        # xlabel='Percentage (%)'
         )
)


In [29]:
coverage_df = (
    bam_df[['chromosomes','arm']]
    .value_counts(normalize=True)
    .mul(100).rename('coverage')
    .reset_index()
)
fig, ax = plt.subplots(figsize=(20,5))
sns.barplot(data=coverage_df, x='chromosomes', y='coverage', hue='arm', ax=ax)
ax.set(
    title='Normalised telomere coverage',
    xlabel='Chromosomes',
    ylabel='Coverage (%)'
)
ax.tick_params(axis='x', labelrotation=45)
ax_settings(ax)


In [49]:
## Average telomere length
mean_telomere_len = bam_df.telomere_len.mean()
median_telomere_len =  bam_df.telomere_len.median()
ax = sns.histplot(data=bam_df, x='telomere_len', kde=True)
ax.axvline(mean_telomere_len, label='mean')
ax.axvline(median_telomere_len, linestyle='--', label='median')
ax.set(
    title = f'Distribution of telomere lengths: Avg {mean_telomere_len:.2f}',
    xlabel ='Telomere lengths (bp)',    
)
ax_settings(ax)


md(f'## Average telmoeric length {mean_telomere_len:,.2}\n\n')


In [32]:
coverage_df_read_count = (
    bam_df[['chromosomes','arm']]
    .value_counts(normalize=False)
    .rename('counts')
    .reset_index()
)

fig, (ax1, ax2) = plt.subplots(2, sharex=True, figsize=(20,10))

sns.barplot(data=coverage_df_read_count, x='chromosomes', y='counts', hue='arm', ax=ax1)


sns.boxplot(
    data=bam_df,
    x='chromosomes',
    y='telomere_len',
    hue='arm',
    ax=ax2
)

ax1.set(
    title='Telomere read counts',
    xlabel='',
    ylabel='Counts'
)

ax2.tick_params(axis='x', labelrotation=45)
ax2.set(
    title='Telomere length per chromosome',
    xlabel='Chromosomes',
    ylabel='Telomeric length (bp)'
)
ax_settings(ax1)
ax_settings(ax2)
